In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
import os
os.chdir('/content/drive/MyDrive/Twitter')

In [4]:
# !apt-get install openjdk-8-jdk-headless
# !wget https://dlcdn.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
# !tar xf spark-3.4.1-bin-hadoop3.tgz

In [5]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285397 sha256=f444f2aa838b32a5127d0b12b3ea75fb638c8345b22d6e5f653a6c9f59d5b6ce
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [6]:
# !pip install findspark

In [30]:
# import findspark
# findspark.init()
import pyspark as ps
import warnings
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
# import pandas as pd
# import pyspark.pandas as ps
import numpy as np
import time

from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer, Word2Vec
from nltk.stem.snowball import SnowballStemmer

from pyspark.ml.classification import LogisticRegression

from pyspark.ml.evaluation import BinaryClassificationEvaluator

**1. Đọc dữ liệu vào**

In [8]:
spark = SparkSession.builder\
        .master("local")\
        .appName('Sentiment Analysis')\
        .getOrCreate()

schema = StructType([\
                    StructField("target", IntegerType(), True),\
                    StructField("id", IntegerType(), True),\
                    StructField("date", DateType(), True),\
                    StructField("flag", StringType(), True),\
                    StructField("user", StringType(), True),\
                    StructField("text", StringType(), True)])

df = spark.read.csv('training.1600000.processed.noemoticon.csv', sep=',', header=False, schema=schema)
df.printSchema()

root
 |-- target: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- flag: string (nullable = true)
 |-- user: string (nullable = true)
 |-- text: string (nullable = true)



In [10]:
df.show(5)

+------+----------+----+--------+---------------+--------------------+
|target|        id|date|    flag|           user|                text|
+------+----------+----+--------+---------------+--------------------+
|     0|1467810369|null|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|     0|1467810672|null|NO_QUERY|  scotthamilton|is upset that he ...|
|     0|1467810917|null|NO_QUERY|       mattycus|@Kenichan I dived...|
|     0|1467811184|null|NO_QUERY|        ElleCTF|my whole body fee...|
|     0|1467811193|null|NO_QUERY|         Karoli|@nationwideclass ...|
+------+----------+----+--------+---------------+--------------------+
only showing top 5 rows



In [11]:
df.head()

Row(target=0, id=1467810369, date=None, flag='NO_QUERY', user='_TheSpecialOne_', text="@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D")

**2. Tiền xử lý**

In [12]:
df.groupBy("target").count().show()

+------+------+
|target| count|
+------+------+
|     4|800000|
|     0|800000|
+------+------+



=> như vậy dữ liệu tích cực (target=4) và tiêu cực (target=0) là cân bằng, không có trường hợp nào target null cần xử lý

Chỉ quan tâm đến các trường dùng để đánh giá (id, target, text)

In [13]:
df1 = df.select("id", "target", "text")

In [14]:
df_clean = df.select('*', (lower(regexp_replace('text', "[^a-zA-Z\\s]", " ")).alias('newtext')))
df_clean.head()

Row(target=0, id=1467810369, date=None, flag='NO_QUERY', user='_TheSpecialOne_', text="@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", newtext=' switchfoot http   twitpic com  y zl   awww  that s a bummer   you shoulda got david carr of third day to do it   d')

In [15]:
tokenizer = Tokenizer(inputCol='newtext', outputCol='tokens')
df_tokens = tokenizer.transform(df_clean)
df_tokens.head()

Row(target=0, id=1467810369, date=None, flag='NO_QUERY', user='_TheSpecialOne_', text="@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", newtext=' switchfoot http   twitpic com  y zl   awww  that s a bummer   you shoulda got david carr of third day to do it   d', tokens=['', 'switchfoot', 'http', '', '', 'twitpic', 'com', '', 'y', 'zl', '', '', 'awww', '', 'that', 's', 'a', 'bummer', '', '', 'you', 'shoulda', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', '', '', 'd'])

In [17]:
remover = StopWordsRemover(inputCol="tokens", outputCol="words_clean")
df_words_no_stopw = remover.transform(df_tokens)
df_words_no_stopw.head()

Row(target=0, id=1467810369, date=None, flag='NO_QUERY', user='_TheSpecialOne_', text="@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", newtext=' switchfoot http   twitpic com  y zl   awww  that s a bummer   you shoulda got david carr of third day to do it   d', tokens=['', 'switchfoot', 'http', '', '', 'twitpic', 'com', '', 'y', 'zl', '', '', 'awww', '', 'that', 's', 'a', 'bummer', '', '', 'you', 'shoulda', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', '', '', 'd'], words_clean=['', 'switchfoot', 'http', '', '', 'twitpic', 'com', '', 'y', 'zl', '', '', 'awww', '', 'bummer', '', '', 'shoulda', 'got', 'david', 'carr', 'third', 'day', '', '', 'd'])

Đưa các token về dạng từ gốc

In [18]:
locale = spark._jvm.java.util.Locale
locale.setDefault(locale.forLanguageTag("en-US"))

stemmer = SnowballStemmer(language='english')
stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens], ArrayType(StringType()))
df_stemmed = df_words_no_stopw.withColumn("words_stemmed", stemmer_udf("words_clean"))
df_stemmed.head()

Row(target=0, id=1467810369, date=None, flag='NO_QUERY', user='_TheSpecialOne_', text="@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", newtext=' switchfoot http   twitpic com  y zl   awww  that s a bummer   you shoulda got david carr of third day to do it   d', tokens=['', 'switchfoot', 'http', '', '', 'twitpic', 'com', '', 'y', 'zl', '', '', 'awww', '', 'that', 's', 'a', 'bummer', '', '', 'you', 'shoulda', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', '', '', 'd'], words_clean=['', 'switchfoot', 'http', '', '', 'twitpic', 'com', '', 'y', 'zl', '', '', 'awww', '', 'bummer', '', '', 'shoulda', 'got', 'david', 'carr', 'third', 'day', '', '', 'd'], words_stemmed=['', 'switchfoot', 'http', '', '', 'twitpic', 'com', '', 'y', 'zl', '', '', 'awww', '', 'bummer', '', '', 'shoulda', 'got', 'david', 'carr', 'third', 'day', '', '', 'd'])

vocab size

Đưa cột "target" về dạng index

In [19]:
label_stringIdx = StringIndexer(inputCol = "target", outputCol = "label")
label_df = label_stringIdx.fit(df_stemmed).transform(df_stemmed)
label_df.show(5)

+------+----------+----+--------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|target|        id|date|    flag|           user|                text|             newtext|              tokens|         words_clean|       words_stemmed|label|
+------+----------+----+--------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
|     0|1467810369|null|NO_QUERY|_TheSpecialOne_|@switchfoot http:...| switchfoot http ...|[, switchfoot, ht...|[, switchfoot, ht...|[, switchfoot, ht...|  0.0|
|     0|1467810672|null|NO_QUERY|  scotthamilton|is upset that he ...|is upset that he ...|[is, upset, that,...|[upset, update, f...|[upset, updat, fa...|  0.0|
|     0|1467810917|null|NO_QUERY|       mattycus|@Kenichan I dived...| kenichan i dived...|[, kenichan, i, d...|[, kenichan, dive...|[, kenichan, dive...|  0.0|
|     0|1467811184|null|NO_QUERY| 

In [20]:
label_df.head()

Row(target=0, id=1467810369, date=None, flag='NO_QUERY', user='_TheSpecialOne_', text="@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", newtext=' switchfoot http   twitpic com  y zl   awww  that s a bummer   you shoulda got david carr of third day to do it   d', tokens=['', 'switchfoot', 'http', '', '', 'twitpic', 'com', '', 'y', 'zl', '', '', 'awww', '', 'that', 's', 'a', 'bummer', '', '', 'you', 'shoulda', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', '', '', 'd'], words_clean=['', 'switchfoot', 'http', '', '', 'twitpic', 'com', '', 'y', 'zl', '', '', 'awww', '', 'bummer', '', '', 'shoulda', 'got', 'david', 'carr', 'third', 'day', '', '', 'd'], words_stemmed=['', 'switchfoot', 'http', '', '', 'twitpic', 'com', '', 'y', 'zl', '', '', 'awww', '', 'bummer', '', '', 'shoulda', 'got', 'david', 'carr', 'third', 'day', '', '', 'd'], label=0.0)

**3. Classification**

3.1. CountVectorizer + IDF + Logistic Regression

In [ ]:
# %%time
# vectorizer = CountVectorizer(minDF=2, inputCol="words_stemmed", outputCol="cntVec")
# cntvec_df = vectorizer.fit(label_df).transform(label_df)
# cntvec_df.show(5)

In [ ]:
# %%time
# idf = IDF(inputCol="cntVec", outputCol="idfVec", minDocFreq=5)
# idf_df = idf.fit(cntvec_df).transform(cntvec_df)
# idf_df.show(5)

In [ ]:
# [train_idf_df, test_idf_df, val] = idf_df.randomSplit([0.1, 0.1, 0.8], seed = 123)

In [ ]:
# %%time
# t1 = time.time()

# lr = LogisticRegression(featuresCol="idfVec")
# lrModel = lr.fit(train_idf_df)
# predictions = lrModel.transform(test_idf_df)

# t2 = time.time()
# print('Time process Logistic Regression: {0:.2f} seconds'.format(t2-t1) )

In [ ]:
# evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")
# evaluator.evaluate(predictions)

In [ ]:
# accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(test_idf_df.count())
# accuracy

3.2. Word2Vec + Logistic Regression

In [39]:
%%time
word2vec = Word2Vec(vectorSize=6, minCount=2, inputCol = "words_stemmed", outputCol = "w2v", seed = 123)
w2v_df = word2vec.fit(label_df).transform(label_df)

CPU times: user 4.02 s, sys: 616 ms, total: 4.63 s
Wall time: 12min 16s


In [40]:
w2v_df.head()

Row(target=0, id=1467810369, date=None, flag='NO_QUERY', user='_TheSpecialOne_', text="@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", newtext=' switchfoot http   twitpic com  y zl   awww  that s a bummer   you shoulda got david carr of third day to do it   d', tokens=['', 'switchfoot', 'http', '', '', 'twitpic', 'com', '', 'y', 'zl', '', '', 'awww', '', 'that', 's', 'a', 'bummer', '', '', 'you', 'shoulda', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', '', '', 'd'], words_clean=['', 'switchfoot', 'http', '', '', 'twitpic', 'com', '', 'y', 'zl', '', '', 'awww', '', 'bummer', '', '', 'shoulda', 'got', 'david', 'carr', 'third', 'day', '', '', 'd'], words_stemmed=['', 'switchfoot', 'http', '', '', 'twitpic', 'com', '', 'y', 'zl', '', '', 'awww', '', 'bummer', '', '', 'shoulda', 'got', 'david', 'carr', 'third', 'day', '', '', 'd'], label=0.0, w2v=DenseVector([-0.1022, -0.4858, 0.0172, 0.2835, -0.0733, -0

In [41]:
[train_w2v_df, test_w2v_df] = w2v_df.randomSplit([0.8, 0.2], seed = 123)

In [42]:
%%time
t1 = time.time()

lrModel_w2v = LogisticRegression(featuresCol="w2v").fit(train_w2v_df)
predictions_w2v = lrModel_w2v.transform(test_w2v_df)

t2 = time.time()
print('Time process Logistic Regression: {0:.2f} seconds'.format(t2-t1) )

Time process Logistic Regression: 360.01 seconds
CPU times: user 2.04 s, sys: 252 ms, total: 2.29 s
Wall time: 6min


In [43]:
evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")
evaluator.evaluate(predictions_w2v)
--0.6510055694515825

0.6510055694515825

In [44]:
accuracy = predictions_w2v.filter(predictions_w2v.label == predictions_w2v.prediction).count() / float(test_w2v_df.count())
accuracy

0.6397333690836589